## Documentation

To read more about synonyms, checkout the docs [here](https://www.elastic.co/guide/en/elasticsearch/reference/current/synonyms-apis.html).

![synonyms_api_docs](../images/synonyms_api_docs.png)

## Connect to ElasticSearch

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'iNEgsrfzSs-A5IWMvnKk8w',
 'name': '5af1aab6c380',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2024-08-05T10:05:34.233336849Z',
             'build_flavor': 'default',
             'build_hash': '1a77947f34deddb41af25e6f0ddb8e830159c179',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '9.11.1',
             'minimum_index_compatibility_version': '7.0.0',
             'minimum_wire_compatibility_version': '7.17.0',
             'number': '8.15.0'}}


## Setting up Synonyms

We are creating an index with a custom analyzer that uses a synonym filter to expand terms like "car" to include "automobile" and "vehicle."

In [ ]:
from pprint import pprint


settings = {
    "settings": {
        "analysis": {
            "filter": {
                "synonym_filter": {
                    "type": "synonym",
                    "synonyms": [
                        "car, automobile, vehicle",
                        "tv, television",
                        "smartphone, mobile, cell phone",
                        "jupyter, jupyter notebook, jupyterlab",
                        # next one is explicit synonym
                        "jupiter, mars, earth, venus, mercury, saturn, uranus, neptune => planet"
                    ]
                }
            },
            "analyzer": {
                "synonym_analyzer": {
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase", # notes: synonyms configurations require lowercase tokens
                        "synonym_filter"
                    ]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "description": {
                "type": "text",
                "analyzer": "synonym_analyzer"
            }
        }
    }
}

index_name = "my_synonym_index"
es.indices.delete(index=index_name, ignore_unavailable=True)
response = es.indices.create(index=index_name, body=settings)
pprint(response.body)

{'acknowledged': True, 'index': 'my_synonym_index', 'shards_acknowledged': True}


## Indexing documents

In [ ]:
import json

from tqdm import tqdm # type: ignore

operations = []
dummy_data = json.load(open("../data/synonyms.json"))
for document in tqdm(dummy_data, total=len(dummy_data)):
    operations.append({'index': {'_index': index_name}})
    operations.append(document)

response = es.bulk(operations=operations)
pprint(response.body)

100%|██████████| 5/5 [00:00<00:00, 76818.75it/s]

{'errors': False,
 'items': [{'index': {'_id': 'd0kBNJUB6odEtf1M-oPS',
                      '_index': 'my_synonym_index',
                      '_primary_term': 1,
                      '_seq_no': 5,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': 'eEkBNJUB6odEtf1M-oPS',
                      '_index': 'my_synonym_index',
                      '_primary_term': 1,
                      '_seq_no': 6,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': 'eUkBNJUB6odEtf1M-oPS',
                      '_index': 'my_synonym_index',
                      '_primary_term': 1,
                      '_seq_no': 7,
                      '_shards': {'failed

## Searching with Synonyms

Now, let’s search for terms that should match synonyms. For example, we’ll search for "vehicle" (which should match "car" or "automobile").

In [11]:
query = {
    "query": {
        "match": {
            "description": "vehicle"
        }
    }
}

response = es.search(index=index_name, body=query)

print("Search Results:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Search Results:
{'description': 'I love my car and television.'}
{'description': 'I love my car and television.'}


In [ ]:
query = {
    "query": {
        "term": { # match will also work
            "description": "planet"
        }
    }
}

response = es.search(index=index_name, body=query)

print("Search Results:")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Search Results:
{'description': 'I want to go to Mars.'}
{'description': 'I want to go to Venus.'}


## Expanding Synonyms for Search-Time Only

If you want to apply synonyms only during search (and not while indexing), you can modify the search query analyzer like this.

In [12]:
settings = {
    "settings": {
        "analysis": {
            "filter": {
                "synonym_filter": {
                    "type": "synonym",
                    "synonyms": [
                        "car, automobile, vehicle",
                        "tv, television"
                    ]
                }
            },
            "analyzer": {
                "index_analyzer": {
                    "tokenizer": "standard",
                    "filter": ["lowercase"]
                },
                "search_analyzer": {
                    "tokenizer": "standard",
                    "filter": ["lowercase", "synonym_filter"]
                }
            }
        }
    },
    "mappings": {
        "properties": {
            "description": {
                "type": "text",
                "analyzer": "index_analyzer",
                "search_analyzer": "search_analyzer"
            }
        }
    }
}

es.indices.delete(index=index_name)
response = es.indices.create(index=index_name, body=settings)
pprint(response.body)

{'acknowledged': True, 'index': 'my_synonym_index', 'shards_acknowledged': True}


In [13]:
import json

from tqdm import tqdm # type: ignore

operations = []
dummy_data = json.load(open("../data/synonyms.json"))
for document in tqdm(dummy_data, total=len(dummy_data)):
    operations.append({'index': {'_index': index_name}})
    operations.append(document)

response = es.bulk(operations=operations)
pprint(response.body)

100%|██████████| 5/5 [00:00<00:00, 30481.86it/s]

{'errors': False,
 'items': [{'index': {'_id': 'fEkDNJUB6odEtf1Ms4MW',
                      '_index': 'my_synonym_index',
                      '_primary_term': 1,
                      '_seq_no': 0,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': 'fUkDNJUB6odEtf1Ms4MW',
                      '_index': 'my_synonym_index',
                      '_primary_term': 1,
                      '_seq_no': 1,
                      '_shards': {'failed': 0, 'successful': 1, 'total': 2},
                      '_version': 1,
                      'result': 'created',
                      'status': 201}},
           {'index': {'_id': 'fkkDNJUB6odEtf1Ms4MW',
                      '_index': 'my_synonym_index',
                      '_primary_term': 1,
                      '_seq_no': 2,
                      '_shards': {'failed

In [14]:
query = {
    "query": {
        "match": {
            "description": "vehicle"
        }
    }
}

response = es.search(index=index_name, body=query)

print("Search Results (Search-time synonyms):")
for hit in response["hits"]["hits"]:
    print(hit["_source"])

Search Results (Search-time synonyms):
{'description': 'I love my car and television.'}
